<a href="https://colab.research.google.com/github/elluy-gabriel-p/Capstone_ML/blob/hapid/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install matplotlib
!pip install pillow


In [2]:
# Clone repository dengan branch tertentu
!git clone --branch hapid hhttps://github.com/elluy-gabriel-p/Capstone_ML

# Periksa file yang telah di-clone
!ls Capstone_ML

# Path dataset
DATASET_PATH = '/Capstone_ML/data/dataset_padang_food'


Cloning into 'Capstone_ML'...
git: 'remote-hhttps' is not a git command. See 'git --help'.

The most similar command is
	remote-https
ls: cannot access 'Capstone_ML': No such file or directory


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameter dataset
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# Data Augmentation dan Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load Data Training dan Validation
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Capstone_ML/data/dataset_padang_food'